<a href="https://colab.research.google.com/github/kaveku/Week1/blob/main/Copy_of_intro_to_wordvectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 2: Introduction to wordvectors </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [146]:
import io, sys
import numpy as np

In [147]:
def load_vectors(filename):
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
    return data

In [148]:
# Loading word vectors
print('')
print(' ** Word vectors ** ')
print('')
'''
word_vectors is a dictionary that maps words to their numerical word vector
[word (string)] = [np-array] 
'''
word_vectors = load_vectors('wiki.en.vec')
print(len(word_vectors))
#print(list(word_vectors.keys())[100])

tree_vector = word_vectors['tree']
print(type(tree_vector), len(tree_vector))


 ** Word vectors ** 

12000
<class 'numpy.ndarray'> 300


In [149]:
## This function computes the cosine similarity between vectors u and v
def cosine(u, v):
  cos = (u.T)@v/(np.linalg.norm(u)* np.linalg.norm(v))
  ## FILL CODE
  return cos 


In [150]:
# compute similarity between words
print('similarity(apple, apples) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['apples']))
print('similarity(apple, banana) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['banana']))
print('similarity(apple, tiger) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['tiger']))

similarity(apple, apples) = 0.637
similarity(apple, banana) = 0.431
similarity(apple, tiger) = 0.212


In [151]:
for i in range (len(word_vectors)):
  j_key = list(word_vectors.keys())[i]


In [152]:
## Functions for nearest neighbor
## This function returns the word corresponding to 
## nearest neighbor vector of x
## The list exclude_words can be used to exclude some
## words from the nearest neighbors search

def nearest_neighbor(x, word_vectors, exclude_words=[]):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    exclude_words (list of strings): words to be excluded from the search
    
    Returns:
    best_word (string) : the word whose word vector is the nearest neighbour 
    to the word vector of x
    '''

    vec = x
    exclude_vecs = []
    for i in range(len(exclude_words)):
      exclude_vecs.append(word_vectors.get(exclude_words[i]))

    best_score = -1.0
    best_word = None

    ## FILL CODE
    for i in range (len(word_vectors)):
      j_key = list(word_vectors.keys())[i]
      line = word_vectors[j_key]
      check = 1
      for j in range(len(exclude_vecs)):
        if (np.array_equal(line, exclude_vecs[j])):
          check = 2
      if check!=2:
        score = cosine(line, vec)
        if (score>best_score):
          best_score = score
          best_word = j_key

    #print(best_word)

    return best_word

In [153]:
print('')
print('The nearest neighbor of cat is: ' , nearest_neighbor(word_vectors['cat'], word_vectors, exclude_words = ['cat', 'cats']))


The nearest neighbor of cat is:  dog


#### Hint (using python priorty queues with the heapq datastructure): 
if you don't want to store all the words and scores you can use the priortiy queue and only store the best K element so far. 

In [154]:

## This function return the words corresponding to the
## K nearest neighbors of vector x.
## You can use the functions heappush and heappop.
def knn(x, vectors, k):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    k (int): number of nearest neighbours to be found
    
    Returns: 
    k_nearest_neighbors (list of tuples): [(score, word), (score, word), ....]
    '''
    x_vec = x
    k_nearest_neighbors = []
    dict_vec ={}

    for key in list(vectors.keys()):
      line= vectors[key]
      dict_vec[key]= cosine(line, x_vec)

    sorted_dt = {key: value for key, value in sorted(dict_vec.items(), key=lambda item: item[1])}

    for i in range(len(sorted_dt)-k, len(sorted_dt)):
      k_nearest_neighbors.append((list(sorted_dt.values())[i], list(sorted_dt.keys())[i]))
        
    return k_nearest_neighbors

In [155]:
knn_cat = knn(word_vectors['cat'], word_vectors, 5)
print('')
print('cat')
print('--------------')
for score, word in knn(word_vectors['cat'], word_vectors, 5):
    print (word, '\t%.3f' % score)



cat
--------------
rabbit 	0.549
pet 	0.573
dog 	0.638
cats 	0.732
cat 	1.000


#### Hint: 
To find the analogies, we find the nearest neighbour associated with the wordvector d
$$ d = \frac{c}{\Vert {c} \Vert} + \frac{b}{\Vert {b} \Vert} - \frac{a}{\Vert {a} \Vert}$$


In [156]:
## This function return the words d, such that a:b and c:d
## verifies the same relation

def analogy(a, b, c, word_vectors):
    '''
    Parameters:
    a (string): word a
    b (string): word b
    c (string): word c
    word_vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    the word d (string) associated with c such that c:d is similar to a:b 
    
    '''
    a_vec = word_vectors.get(a)
    b_vec = word_vectors.get(b)
    c_vec = word_vectors.get(c)

    d = (c_vec/np.linalg.norm(c_vec))+(b_vec/np.linalg.norm(b_vec))-(a_vec/np.linalg.norm(a_vec))

    vec = d

    return(nearest_neighbor(vec, word_vectors, exclude_words=[]))

In [157]:
# Word analogies
print('')
print('france - paris + rome = ' , analogy('paris', 'france', 'rome', word_vectors))


france - paris + rome =  italy


## A word about biases in word vectors

In [158]:
## A word about biases in word vectors:
print('')
print('similarity(genius, man) = %.3f' %
      cosine(word_vectors['man'], word_vectors['genius']))
print('similarity(genius, woman) = %.3f' %
      cosine(word_vectors['woman'], word_vectors['genius']))


similarity(genius, man) = 0.445
similarity(genius, woman) = 0.325


In [159]:
## Compute the association strength between:
##   - a word w
##   - two sets of attributes A and B

def association_strength(w, A, B, vectors):
    '''
    Parameters:
    w (string): word w
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    Returnrs: 
    strength (float): the value of the association strength 
    '''
    w = word_vectors.get(w)
    strength = 0.0
    part_a = 0.0
    part_b = 0.0 
    
    ## FILL CODE
    #strength = (1/len(A))*np.sum(np.cos(w, A)) - (1/len(B))*np.sum(np.cos(w, B))
    sum_a = 0
    for i in range(len(A)):
      set_w = word_vectors.get(A[i])
      #word_main = word_vectors.get(w)
      sim_i = cosine(set_w, w)
      sum_a = sum_a+sim_i

    sum_b = 0
    for i in range(len(B)):
      set_w = word_vectors.get(B[i])
      #word_main = word_vectors.get(w)
      sim_i = cosine(set_w, w)
      sum_b = sum_b+sim_i


      strength = (1/len(A))*sum_a - (1/len(B))*sum_b

    return strength

In [160]:
## Perform the word embedding association test between:
##   - two sets of words X and Y
##   - two sets of attributes A and B

def weat(X, Y, A, B, vectors):
  
    '''
    Parameters:
    X (list of strings): The words belonging to set X
    Y (list of strings): The words belonging to set Y
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returns: 
    score (float): the value of the group association strength  
    '''
    print(X,Y)
    score_a = 0.0
    score_b = 0.0
    for i in range(len(X)):
      score_a = score_a+(association_strength(X[i], A, B, vectors))
    for i in range(len(Y)):
      score_b = score_b+(association_strength(Y[i], A, B, vectors))
    ## FILL CODE

    return score_a - score_b

In [161]:
## Replicate one of the experiments from:
##
## Semantics derived automatically from language corpora contain human-like biases
## Caliskan, Bryson, Narayanan (2017)

career = ['executive', 'management', 'professional', 'corporation', 
          'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family',
          'cousins', 'marriage', 'wedding', 'relatives']
male = ['john', 'paul', 'mike', 'kevin',   'steve', 'greg', 'jeff', 'bill']
female = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print('')
print('Word embedding association test: %.3f' %weat(career, family, male, female, word_vectors))


['executive', 'management', 'professional', 'corporation', 'salary', 'office', 'business', 'career'] ['home', 'parents', 'children', 'family', 'cousins', 'marriage', 'wedding', 'relatives']
Word embedding association test: 0.847


## Word translation using word vectors

In the following, we will use word vectors in English and French to translate words from English to French. The idea is to learn a linear function that maps English word vectors to their correponding French word vectors. To learn this linear mapping, we will use a small bilingual lexicon, that contains pairs of words in English and French that are translations of each other.

The following function will load the small English-French bilingual lexicon:

In [162]:
def load_lexicon(filename):
    '''
    Parameters:
    filename(string): the path of the lexicon
    
    Returns:
    data(list of pairs of string): the bilingual lexicon
    '''
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    data = []
    for line in fin:
        a, b = line.rstrip().split(' ')
        data.append((a, b))
    return data

In [163]:
word_vectors_en = load_vectors('wiki.en.vec')
word_vectors_fr = load_vectors('wiki.fr.vec')
lexicon = load_lexicon("lexicon-en-fr.txt")
print(lexicon[:5])

[('the', 'le'), ('the', 'les'), ('the', 'la'), ('and', 'et'), ('was', 'fut')]


In [164]:
# We split the lexicon into a train and validation set
train = lexicon[:5000]
valid = lexicon[5000:5100]

The following function will learn the mapping from English to French. The idea is to build two matrices $X_{\text{en}}$ and $X_{\text{fr}}$, and to find a mapping $M$ that minimizes $||X_{\text{en}} W - X_{\text{fr}} ||_2$. In numpy, this mapping can be obtained by using the `numpy.linalg.lstsq` function.

In [165]:
def align(word_vectors_en, word_vectors_fr, lexicon):
    '''
    Parameters:
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    lexicon(list of pairs of string): bilingual training lexicon
    
    Returns
    mapping(np.array): the mapping from English to French vectors
    '''
    x_en, x_fr = [], []
    for i in range(len(lexicon)):
      word_en = lexicon[i][0]
      word_fr = lexicon[i][1]
      x_en.append(word_vectors_en.get(word_en))
      x_fr.append(word_vectors_fr.get(word_fr))
    x_en = np.array(x_en)
    x_fr = np.array(x_fr)

    
    ## FILL CODE
    
    return np.linalg.lstsq(x_en, x_fr)[0]

In [166]:
mapping = align(word_vectors_en, word_vectors_fr, lexicon)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Given a mapping, a set of word English word vector and French word vectors, the next function will translate the English word to French. To do so, we apply the mapping on the English word, and retrieve the nearest neighbor of the obtained vector in the set of French word vectors. The translation is then the corresponding French word.

In [167]:
def translate(word, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    word(string): an English word
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    A string containing the translation of the English word
    '''
    word_vec = word_vectors_en.get(word)
    word_vec_mapped = word_vec@mapping

    

    return nearest_neighbor(word_vec_mapped, word_vectors_fr, exclude_words=[])

In [168]:
print(translate("world", word_vectors_en, word_vectors_fr, mapping))
print(translate("machine", word_vectors_en, word_vectors_fr, mapping))
print(translate("learning", word_vectors_en, word_vectors_fr, mapping))

monde
machine
apprentissage


Finally, let's implement a function to evaluate this method on the validation lexicon:

In [169]:
def evaluate(valid, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    valid(a list of pairs of string): the validation lexicon
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    Accuracy(float): the accuracy on the validation lexicon
    '''
    acc = 0.0
    true_vals =[]
    for i in range(len(valid)):
      french_trans = translate(valid[i][0], word_vectors_en, word_vectors_fr, mapping)
      if(french_trans==valid[i][1]):
        true_vals.append(french_trans)
    acc = len(true_vals)/len(valid)

    return acc

In [170]:
evaluate(valid, word_vectors_en, word_vectors_fr, mapping)

0.64